In [74]:
import pandas as pd
import numpy as np
import scipy.io
import mat73
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, LeakyReLU, BatchNormalization
from tensorflow.keras.utils import plot_model, to_categorical
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.svm import SVC as svm
from sklearn.linear_model import LogisticRegression as lg
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score,balanced_accuracy_score,f1_score
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold
from sklearn.exceptions import ConvergenceWarning
from warnings import simplefilter,filterwarnings
import os
# ignore all future warnings1
simplefilter(action='ignore', category=FutureWarning)


## EEG data:

In [84]:
 ## data loading
data = mat73.loadmat('x_source.mat')  
id = scipy.io.loadmat('id.mat')
IDs = pd.DataFrame(id["id"])
#flattening and adding id
df2 = pd.DataFrame(data['x'].reshape((data['x'].shape[0], -1)))
df2['IDs'] = IDs
df2.shape

(2041, 26589)

In [95]:
df2['IDs']#array of arrays
df2['IDs'][0][0] #this is what i want for 1 guy
df2['IDs'] = [df2['IDs'][i][0] for i in range(2041)]
df2.shape


(2041, 26589)

## Behavioral Data:


In [89]:
behaviour_data = pd.read_csv('scores.csv')

### We need columns with:
1. IDs

 
2. for Anxiety:  "SCARED_SR" "SCARED_P". 
    


2. for ADHD inattention: Strengths and Weaknesses of ADHD Symptoms and Normal Behavior Scale ("SWAN IN")


3. for ADHD combined: Strengths and Weaknesses of ADHD Symptoms and Normal Behavior Scale ("SWAN HY")


4. for Autism Social Responsiveness Scale-2 ("SRS_Pre" (preschool) or "SRS") 


5. for Depression: Children's Depression Index (average score of: "CDI_SR" "CDI_P") (alternative 1: Mood & Feelings Questionnaire (MFQ); Alternative 2: Affective Reactivity Index—(ARI-S) Self Report)


6. WISC-Full IQ "WISC_FSIQ"


In [91]:
behaviour_data = behaviour_data[['IDs', 'MFQ_P_Total', 'SCARED_P_Total', 'SWAN_HY', "SWAN_IN", 'SRS_Total','ARI_S_Total_Score', 'WISC_FSIQ' ]]
behaviour_data.shape

(3076, 8)

In [92]:
def missing_values_table(df):
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values'})
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
    '% of Total Values', ascending=False).round(1)
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns (with column 'IDs').\n"      
        "There are " + str(mis_val_table_ren_columns.shape[0]) +
            " columns that have missing values.")
    return mis_val_table_ren_columns

missing_values_table(behaviour_data)

Your selected dataframe has 8 columns (with column 'IDs').
There are 7 columns that have missing values.


,Missing Values,% of Total Values
WISC_FSIQ,588,19.1
MFQ_P_Total,466,15.1
SCARED_P_Total,328,10.7
SRS_Total,218,7.1
SWAN_HY,163,5.3
SWAN_IN,163,5.3
ARI_S_Total_Score,125,4.1


## Merge EEG and behavioral data together:

In [93]:
df = pd.merge(df2, behaviour_data, on='IDs', how='inner')
df.shape

(1929, 26596)

### features and labels preparation:

In [97]:
Y = df[[ 'MFQ_P_Total', 'SCARED_P_Total', 'SWAN_HY', "SWAN_IN", 'SRS_Total','ARI_S_Total_Score', 'WISC_FSIQ' ]]


,MFQ_P_Total,SCARED_P_Total,SWAN_HY,SWAN_IN,SRS_Total,ARI_S_Total_Score,WISC_FSIQ
0,NaN,NaN,-2.444444,-2.111111,4.0,5.0,140.0
1,NaN,NaN,NaN,NaN,NaN,6.0,NaN
2,NaN,NaN,0.555555,-1.111111,52.0,1.0,88.0
3,28.0,13.0,1.333333,1.555555,91.0,7.0,87.0
4,2.0,2.0,0.444444,0.555555,18.0,7.0,95.0
...,...,...,...,...,...,...,...
1924,3.0,25.0,-2.444444,-2.777777,12.0,0.0,111.0
1925,34.0,32.0,0.666666,2.222222,37.0,5.0,115.0
1926,0.0,17.0,0.000000,0.000000,22.0,5.0,NaN
1927,22.0,12.0,-1.444444,1.333333,36.0,1.0,104.0


In [94]:
print("X:", X.shape, "Y:", Y.shape)

NameError: name 'X' is not defined